# RAG 文档问答系统

本 Notebook 实现一个基于 **向量检索增强生成 (RAG)** 的文档问答系统。支持加载多种格式的文档（TXT/PDF/Word/Markdown），通过向量检索找到相关内容，结合大语言模型生成准确的回答。

## 1. 文档加载

支持的文件格式：**TXT / PDF / Word (.docx) / Markdown (.md)**

把你的文件放到 `knowledge_base/docs/` 目录下，运行下面的代码即可加载。

In [4]:
# !pip install langchain langchain-community langchain-text-splitters pypdf docx2txt

import os
from pathlib import Path
from langchain_community.document_loaders import TextLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# =============================================================================
# 配置：文档目录
# =============================================================================
DOCS_PATH = "knowledge_base/docs"
os.makedirs(DOCS_PATH, exist_ok=True)

# =============================================================================
# 加载所有格式的文件（带容错处理）
# =============================================================================
def load_all_documents(docs_path: str):
    """加载目录下所有支持格式的文件（跳过损坏文件）"""
    all_docs = []
    failed_files = []
    
    # TXT 文件
    for file_path in Path(docs_path).rglob("*.txt"):
        try:
            loader = TextLoader(str(file_path), encoding='utf-8')
            all_docs.extend(loader.load())
        except Exception as e:
            failed_files.append((str(file_path), str(e)[:50]))
    
    # PDF 文件（逐个加载，跳过损坏的）
    try:
        from langchain_community.document_loaders import PyPDFLoader
        for file_path in Path(docs_path).rglob("*.pdf"):
            try:
                loader = PyPDFLoader(str(file_path))
                all_docs.extend(loader.load())
            except Exception as e:
                failed_files.append((str(file_path), str(e)[:50]))
    except ImportError:
        print("  ⚠️ pypdf 未安装，跳过 PDF 文件")
    
    # Word 文件
    for file_path in Path(docs_path).rglob("*.docx"):
        try:
            loader = Docx2txtLoader(str(file_path))
            all_docs.extend(loader.load())
        except Exception as e:
            failed_files.append((str(file_path), str(e)[:50]))
    
    # Markdown 文件
    for file_path in Path(docs_path).rglob("*.md"):
        try:
            loader = TextLoader(str(file_path), encoding='utf-8')  # 简单用 TextLoader
            all_docs.extend(loader.load())
        except Exception as e:
            failed_files.append((str(file_path), str(e)[:50]))
    
    # 统计
    txt_count = len(list(Path(docs_path).rglob("*.txt")))
    pdf_count = len(list(Path(docs_path).rglob("*.pdf")))
    docx_count = len(list(Path(docs_path).rglob("*.docx")))
    md_count = len(list(Path(docs_path).rglob("*.md")))
    
    if txt_count: print(f"  ✅ TXT: {txt_count} 个")
    if pdf_count: print(f"  ✅ PDF: {pdf_count} 个")
    if docx_count: print(f"  ✅ Word: {docx_count} 个")
    if md_count: print(f"  ✅ Markdown: {md_count} 个")
    
    if failed_files:
        print(f"\n  ⚠️ {len(failed_files)} 个文件加载失败（已跳过）:")
        for f, err in failed_files[:5]:
            print(f"    - {Path(f).name}: {err}")
        if len(failed_files) > 5:
            print(f"    ... 还有 {len(failed_files)-5} 个")
    
    return all_docs

print(f"📁 文档目录: {os.path.abspath(DOCS_PATH)}\n")
documents = load_all_documents(DOCS_PATH)

if documents:
    print(f"\n✅ 共加载 {len(documents)} 个文档片段")
    
    # 显示文件列表
    sources = sorted(set(doc.metadata.get('source', '未知') for doc in documents))
    print(f"\n📄 成功加载的文件 ({len(sources)} 个):")
    for src in sources[:10]:
        print(f"  - {Path(src).name}")
    if len(sources) > 10:
        print(f"  ... 还有 {len(sources) - 10} 个")
else:
    print(f"⚠️ 没有成功加载任何文件")

📁 文档目录: /home/wangct/homework/text2sql/sqlmodel/rag/knowledge_base/docs



Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 50 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 56 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 78 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)
Ignoring wrong pointing object 86 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 90 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 94 0 (offset 0)
Ignoring wrong pointing object 96 0 (offset 0)
Ignoring wrong pointing object 98 0 (offset 0)
Ignoring wrong

  ✅ TXT: 1 个
  ✅ PDF: 88 个
  ✅ Word: 10 个

✅ 共加载 7317 个文档片段

📄 成功加载的文件 (99 个):
  - 01 Intro.pdf
  - 02 Data_Warehouse.pdf
  - 03 Preprocessing.pdf
  - 04 classification.pdf
  - 05 ARM.pdf
  - 06 clustering.pdf
  - 1+Chapter+1+Introduction.pdf
  - 1+Chapter+2+Bayesian+Learning+.pdf
  - 10+Chapter+5+Regression.pdf
  - 11+Chapter+7+Feature+Selection.pdf
  ... 还有 89 个


## 1.2 向量数据库管理（查看/删除/清空）

当你删除了本地文件，也需要从向量数据库中删除对应的向量，否则会：
- 数据库越来越大
- 检索到已删除文件的内容（脏数据）

In [ ]:
# =============================================================================
# 向量数据库管理工具
# =============================================================================

def list_indexed_files():
    """查看向量数据库中有哪些文件"""
    if not vectordb:
        print("⚠️ 向量数据库未初始化")
        return []
    
    # 获取所有文档的元数据
    collection = vectordb._collection
    result = collection.get(include=["metadatas"])
    
    # 统计每个文件的向量数量
    file_stats = {}
    for meta in result['metadatas']:
        source = meta.get('source', '未知')
        file_stats[source] = file_stats.get(source, 0) + 1
    
    print(f"📊 向量数据库统计：共 {len(result['ids'])} 个向量，来自 {len(file_stats)} 个文件\n")
    for file, count in sorted(file_stats.items()):
        print(f"  📄 {file} ({count} 个向量)")
    
    return list(file_stats.keys())

def delete_file_vectors(file_path: str):
    """删除指定文件的所有向量"""
    if not vectordb:
        print("⚠️ 向量数据库未初始化")
        return
    
    collection = vectordb._collection
    result = collection.get(include=["metadatas"])
    
    # 找出要删除的向量 ID
    ids_to_delete = []
    for i, meta in enumerate(result['metadatas']):
        source = meta.get('source', '')
        # 支持部分匹配（文件名或完整路径）
        if file_path in source or source.endswith(file_path):
            ids_to_delete.append(result['ids'][i])
    
    if ids_to_delete:
        collection.delete(ids=ids_to_delete)
        print(f"✅ 已删除 {len(ids_to_delete)} 个向量（来自: {file_path}）")
    else:
        print(f"⚠️ 未找到文件: {file_path}")

def clear_all_vectors():
    """清空整个向量数据库"""
    import shutil
    persist_dir = 'chroma_db_langchain'
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)
        print("🗑️ 向量数据库已清空")
    else:
        print("⚠️ 向量数据库不存在")

def sync_with_local_files(docs_path: str):
    """同步：删除向量数据库中已不存在的本地文件"""
    if not vectordb:
        print("⚠️ 向量数据库未初始化")
        return
    
    # 获取本地文件列表
    local_files = set()
    for ext in ['*.txt', '*.pdf', '*.docx', '*.md']:
        local_files.update(str(f) for f in Path(docs_path).rglob(ext))
    
    # 获取向量数据库中的文件列表
    collection = vectordb._collection
    result = collection.get(include=["metadatas"])
    
    indexed_files = set(meta.get('source', '') for meta in result['metadatas'])
    
    # 找出需要删除的（在向量库中但本地已删除）
    to_delete = indexed_files - local_files
    
    if to_delete:
        print(f"🔍 发现 {len(to_delete)} 个已删除的文件，正在清理向量...")
        for file in to_delete:
            delete_file_vectors(file)
        print("✅ 同步完成")
    else:
        print("✅ 向量数据库与本地文件已同步，无需清理")

# =============================================================================
# 使用示例
# =============================================================================
print("📋 向量数据库管理命令：")
print("  list_indexed_files()           # 查看所有已索引的文件")
print("  delete_file_vectors('文件名')   # 删除指定文件的向量")
print("  sync_with_local_files(MY_DOCS_PATH)  # 自动同步（删除已移除文件的向量）")
print("  clear_all_vectors()            # 清空整个数据库")
print()

# 查看当前状态
if 'vectordb' in dir() and vectordb:
    list_indexed_files()

## 2. 嵌入与向量存储 (LangChain + ChromaDB)

嵌入模型（Embedding Model）的质量和向量数据库的效率直接影响检索的准确性和速度。

**任务**:
- 使用 LangChain 和 ChromaDB 将文档切片并向量化入库。

In [ ]:
# !pip install langchain langchain-community langchain-text-splitters chromadb

import os
import re
import shutil
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import DashScopeEmbeddings
import uuid

# =============================================================================
# 配置
# =============================================================================
PERSIST_DIR = 'chroma_db_rag'  # 换一个新目录名
FORCE_REBUILD = True  # 首次使用设为 True

# =============================================================================
# 文本清理函数（移除无效 Unicode 字符）
# =============================================================================
def clean_text(text: str) -> str:
    """清理文本中的无效字符"""
    text = text.encode('utf-8', errors='ignore').decode('utf-8')
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\x9f]', '', text)
    return text

# =============================================================================
# 初始化嵌入模型（使用 DashScope API）
# =============================================================================
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")
if not DASHSCOPE_API_KEY:
    print("❌ 请先设置 DASHSCOPE_API_KEY 环境变量")
    embed_model = None
else:
    try:
        embed_model = DashScopeEmbeddings(model="text-embedding-v2", dashscope_api_key=DASHSCOPE_API_KEY)
        test_emb = embed_model.embed_query("测试")
        print(f"✅ 嵌入模型: DashScope text-embedding-v2")
        print(f"   向量维度: {len(test_emb)}")
    except Exception as e:
        print(f"❌ 嵌入模型加载失败: {e}")
        embed_model = None

# =============================================================================
# 加载或创建向量数据库
# =============================================================================
vectordb = None
retriever = None

if embed_model:
    db_exists = os.path.exists(PERSIST_DIR) and os.path.exists(os.path.join(PERSIST_DIR, 'chroma.sqlite3'))
    has_documents = 'documents' in dir() and documents
    
    if db_exists and not FORCE_REBUILD:
        # 加载已有数据库
        print(f"\n📂 加载已有向量数据库...")
        try:
            vectordb = Chroma(persist_directory=PERSIST_DIR, embedding_function=embed_model)
            count = vectordb._collection.count()
            if count > 0:
                print(f"✅ 已加载 {count} 个向量")
                retriever = vectordb.as_retriever(search_kwargs={"k": 5})
                print("✅ 检索器已就绪！")
            else:
                print(f"⚠️ 数据库为空")
                db_exists = False
        except Exception as e:
            print(f"⚠️ 加载失败: {e}")
            db_exists = False
    
    if (not db_exists or FORCE_REBUILD) and has_documents:
        print(f"\n📝 正在创建向量数据库...")
        
        # 清理文档文本
        print("   清理文档中的无效字符...")
        for doc in documents:
            doc.page_content = clean_text(doc.page_content)
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=30)
        docs = text_splitter.split_documents(documents)
        print(f"   文档切分为 {len(docs)} 个文本块")
        
        # 删除旧数据库（如果存在）
        if os.path.exists(PERSIST_DIR):
            try:
                shutil.rmtree(PERSIST_DIR)
                print(f"   已删除旧数据库")
            except Exception as e:
                print(f"   ⚠️ 删除旧数据库失败: {e}")
                # 换一个目录名
                PERSIST_DIR = f'chroma_db_rag_{uuid.uuid4().hex[:8]}'
                print(f"   使用新目录: {PERSIST_DIR}")
        
        ids = [str(uuid.uuid4()) for _ in docs]
        vectordb = Chroma.from_documents(
            documents=docs, 
            embedding=embed_model,
            ids=ids,
            persist_directory=PERSIST_DIR
        )
        print(f"✅ 向量数据库已创建，共 {len(docs)} 个向量")
        print(f"   保存位置: {os.path.abspath(PERSIST_DIR)}")
        retriever = vectordb.as_retriever(search_kwargs={"k": 5})
        print("✅ 检索器已就绪！")
    
    elif not has_documents and (not db_exists or FORCE_REBUILD):
        print("\n❌ 数据库为空，且没有加载文档")
        print("   👉 请先运行【模块 1】加载文档，然后再运行本模块")

✅ 嵌入模型: DashScope text-embedding-v2
   向量维度: 1536

📝 正在创建向量数据库...
   清理文档中的无效字符...
   文档切分为 13764 个文本块
✅ 向量数据库已创建，共 13764 个向量
   保存位置: /home/wangct/homework/text2sql/sqlmodel/rag/chroma_db_rag
✅ 检索器已就绪！


## 3. RAG 问答实现

使用通用提示词模板实现 RAG 问答功能：
- 向量检索相关文档
- 构建带约束的提示词
- 调用 LLM 生成回答（带出处标注）

In [ ]:
# --- 0. 检查 LLM 客户端 ---
# ⚠️ 请在环境变量中设置 DASHSCOPE_API_KEY，或在 .env 文件中配置
if 'client' not in locals() or client is None:
    try:
        from openai import OpenAI
        
        # 从环境变量获取 API Key
        DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", None)
        if not DASHSCOPE_API_KEY:
            print("⚠️ 警告：未设置 DASHSCOPE_API_KEY 环境变量！")
            print("   请运行: export DASHSCOPE_API_KEY='your-api-key-here'")
            print("   或在 .env 文件中添加: DASHSCOPE_API_KEY=your-api-key-here")
        
        client = OpenAI(
            api_key=DASHSCOPE_API_KEY,
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        print("✅ 已初始化 DashScope client。")
    except Exception as e:
        print(f"❌ 初始化 DashScope client 失败: {e}")
        client = None
else:
    print("✅ 复用已有的 DashScope client。")

# --- 配置使用的模型 ---
LLM_MODEL = "qwen-turbo"  # 可选: "qwen-plus", "qwen-turbo", "qwen-max"

# =============================================================================
# 通用 RAG 提示词模板
# =============================================================================
RAG_PROMPT_TEMPLATE = """
你是一个专业、严谨的知识库助手。请基于提供的【参考信息】来回答用户的【问题】。

【重要约束】：
1. 仅根据已知的【参考信息】回答，严禁基于外部知识编造内容
2. 如果【参考信息】中不包含答案，请直接告知用户"根据现有资料无法回答该问题"
3. 禁止产生幻觉，不确定的内容不要回答
4. 回答要简洁、准确
5. 如果引用了文档内容，请在回答末尾标注出处

【参考信息】：
{context}

【问题】：
{query}

【回答】：
"""

# =============================================================================
# RAG 问答函数
# =============================================================================
def rag_ask(question: str):
    """
    RAG 问答函数：基于向量检索的问答
    
    Args:
        question: 用户问题
    Returns:
        str: 回答
    """
    if not client:
        return "❌ LLM 客户端未初始化"
    if not retriever:
        return "❌ 检索器未初始化，请先运行模块 2"
    
    # 1. 向量检索
    docs = retriever.invoke(question)
    
    # 2. 格式化上下文（带来源）
    context_parts = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', '未知')
        context_parts.append(f"[文档{i}] (来源: {source})\n{doc.page_content}")
    context = "\n\n".join(context_parts) if context_parts else "无相关文档"
    
    # 3. 构建提示词并调用 LLM
    prompt = RAG_PROMPT_TEMPLATE.format(context=context, query=question)
    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

# =============================================================================
# 测试示例
# =============================================================================
print("=" * 60)
print(f"📚 RAG 问答系统 (模型: {LLM_MODEL})")
print("=" * 60)
print("\n💡 使用方式：")
print("  rag_ask('你的问题')")

if client and retriever:
    print("\n✅ 系统已就绪，可以开始提问！")
else:
    missing = []
    if not client: missing.append("LLM客户端")
    if not retriever: missing.append("检索器")
    print(f"\n⚠️ 缺少组件: {', '.join(missing)}")

✅ 已初始化 DashScope client。
📚 RAG 问答系统 (模型: qwen-turbo)

💡 使用方式：
  rag_ask('你的问题')

📝 示例问题：
  - rag_ask('文档中提到了哪些主题？')
  - rag_ask('请总结一下主要内容')

✅ 系统已就绪，可以开始提问！


In [18]:
a = rag_ask('课程和先修课程之间是什么关系？')
print(a)

课程和先修课程之间的关系是通过“prereq”表来定义的，其中每个课程可能有多个先修课程。具体来说，一个课程的先修课程是该课程学习之前必须完成的其他课程。这种关系由主键（course_id, prereq_id）以及外键关系（course_id → course.course_id，prereq_id → course.course_id）来维护。 

出处：[文档3]
